In [ ]:
#pip install wordcloud

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
#from wordcloud import WordCloud
from gensim.models import Word2Vec
from nltk import word_tokenize, FreqDist
import PIL
from collections import Counter
import itertools
import gc
sns.set()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from google.colab import files
files.upload()

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d virenkathiriya/totaldata
!kaggle datasets download -d virenkathiriya/politifact
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

import zipfile
with zipfile.ZipFile('./fake-and-real-news-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

with zipfile.ZipFile('./totaldata.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

total=pd.read_csv('NewsDataSet.csv')

politi = pd.read_csv('Politifact_.csv')
fake=pd.read_csv('Fake.csv')
category=['FALSE']*(23481)
fake['Category']=category
fake.head() 
true=pd.read_csv('True.csv')
category=['TRUE']*(21417)
true['Category']=category
true.head()
true=true.head(5000)
fake=fake.head(5000)
kaggle=true.head(5000).append(fake.head(5000),ignore_index=True)

true

df1 = pd.read_csv('fakenewsnet_kaggle/BuzzFeed_fake_news_content.csv', delimiter=',')
df1.dataframeName = 'BuzzFeed_fake_news_content.csv'


,text,Category
0,philippine president rodrigo duterte warned th...,0
1,pope francis recently canceled some engagement...,0
2,"By now, the whole world knows that the investi...",0
3,Donald Trump s Muslim ban is proving to be a l...,0
4,WASHINGTON (Reuters) - The U.S. attorney’s off...,1
...,...,...
13697,"If Republicans listen to this sage advice, Don...",0
13698,"On Tuesday, Donald Trump announced the identit...",0
13699,You are using an older browser version. Please...,0
13700,NEW YORK (Reuters) - Puerto Rico’s federal ove...,1


In [ ]:

df2 = pd.read_csv('fakenewsnet_kaggle/BuzzFeed_real_news_content.csv', delimiter=',')
df2.dataframeName = 'BuzzFeed_real_news_content.csv'


In [ ]:
df1.text.iloc[88]

In [ ]:
df3 = pd.read_csv('fakenewsnet_github/politifact_fake_updated.csv', delimiter=',')
df4 = pd.read_csv('fakenewsnet_github/politifact_real_updated.csv', delimiter=',')
df5 = pd.read_csv('fakenewsnet_github/gossipcop_fake_updated.csv', delimiter=',')
df6 = pd.read_csv('fakenewsnet_github/gossipcop_real_updated.csv', delimiter=',')

In [ ]:
df1 =df1.dropna(subset=['title', 'text'])
df2 =df2.dropna(subset=['title', 'text'])
df3 =df3.dropna(subset=['title', 'text'])
df4 =df4.dropna(subset=['title', 'text'])
df5 =df5.dropna(subset=['title', 'text'])
df6 =df6.dropna(subset=['title', 'text'])

In [ ]:
df1 = df1.reset_index()
df2 = df2.reset_index()
df3 = df3.reset_index()
df4 = df4.reset_index()
df5 = df5.reset_index()
df6 = df6.reset_index()

In [ ]:
dflistreal = [df2,df4,df6]
dflistfake = [df1,df3,df5]
dflistreal=[true]
dflistfake=[fake]

In [ ]:
for df in dflistreal:
    df['target'] = 0
    df['label'] = 'Real'
for df in dflistfake:
    df['target'] = 1
    df['label'] = 'Fake'    

total['target']=0
total['label']='Fake'
for i in range(len(total)):
  if(total.at[i, 'Category']==1):
    total.at[i, 'target']=1
    total.at[i,'label'] = 'Real'
  elif(total.at[i, 'Category']==0):
    total.at[i, 'target']=0
    total.at[i,'label'] = 'Fake'

total

,text,Category,target,label
0,philippine president rodrigo duterte warned th...,0,0,Fake
1,pope francis recently canceled some engagement...,0,0,Fake
2,"By now, the whole world knows that the investi...",0,0,Fake
3,Donald Trump s Muslim ban is proving to be a l...,0,0,Fake
4,WASHINGTON (Reuters) - The U.S. attorney’s off...,1,1,Real
...,...,...,...,...
13697,"If Republicans listen to this sage advice, Don...",0,0,Fake
13698,"On Tuesday, Donald Trump announced the identit...",0,0,Fake
13699,You are using an older browser version. Please...,0,0,Fake
13700,NEW YORK (Reuters) - Puerto Rico’s federal ove...,1,1,Real


In [ ]:
df_merge = pd.concat([fake, true], sort = 'False')

In [ ]:
df_merge = df_merge.drop(columns=['id', 'index', 'Unnamed: 0', 'level_0','summary'], axis =1)

KeyError: ignored

In [ ]:
df_merge
df_merge = df_merge.sample(frac=1).reset_index(drop=True)

,Category,date,label,subject,target,text,title
0,TRUE,"April 7, 2017",Real,politicsNews,0,BERLIN (Reuters) - The U.S. military strike on...,German foreign minister says U.S. strike on Sy...
1,FALSE,"May 16, 2017",Fake,News,1,"Comedian Dave Chappelle, known for his razor-s...",Dave Chappelle: ‘I F***ed Up’ When Asking To ...
2,TRUE,"April 10, 2017",Real,politicsNews,0,WASHINGTON (Reuters) - U.S. Secretary of State...,Tillerson faces tough talks in Moscow amid inc...
3,FALSE,"October 21, 2016",Fake,News,1,"Curt Schilling, who is challenging Elizabeth W...",Curt Schilling Wants Elizabeth Warren’s Seat ...
4,FALSE,"January 15, 2017",Fake,News,1,"It took Donald Trump all day, but he apparentl...",Trump Finally Watched Alec Baldwin Mock His R...
...,...,...,...,...,...,...,...
9995,TRUE,"December 13, 2017",Real,politicsNews,0,(Reuters) - Senior FBI officials who helped pr...,FBI officials said Clinton 'has to win' race t...
9996,FALSE,"February 12, 2017",Fake,News,1,Trump senior adviser Stephen Miller told ABC s...,The Internet Rips Trump Adviser’s Face Off Af...
9997,FALSE,"December 7, 2016",Fake,News,1,A Carrier union leader called Donald Trump out...,Trump Attacks Union Leader Who Called Him Out...
9998,TRUE,"July 31, 2017",Real,politicsNews,0,WASHINGTON (Reuters) - A new U.S. rule aimed a...,New U.S. rule on class actions survives first ...


In [ ]:
del fake, true;
gc.collect();

In [ ]:
df_merge=total
df_merge['combined_text'] = df_merge.text
df_merge['combined_text'] = df_merge.title + ' ' + df_merge.text

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

stopwords_list = stopwords.words('english') + list(string.punctuation)

# It is generally a good idea to also remove punctuation



# Now we have a list that includes all english stopwords, as well as all punctuation
stopwords_list += ["''", '""', '...', '``','”', "`","*", "’", "“", "‘", "–", "⋆", "'s", "—", "well"]
stopwords_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
text_cols = ['title', 'text']
target = df_merge['target']
#cat_cols = ['source']
#used_cols = text_cols 
#X = df_merge[used_cols]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_merge['combined_text'] ,target, test_size=0.30, random_state = 45)

In [ ]:
del df_merge;
gc.collect();

In [ ]:
X_train.shape
y_train.shape

(9591,)

In [ ]:
y_train

8110     0
8212     0
5657     0
7592     1
575      0
        ..
12895    1
11808    0
6012     1
6558     0
7115     1
Name: target, Length: 9591, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
!pip install category_encoders
import category_encoders as ce

from time import time
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
'''# Preprocessing for text
text_transformer = Pipeline(steps =[('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer())
])

# Preprocessing for categorical data
#categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
#    ('loo', ce.leave_one_out.LeaveOneOutEncoder())
#])   
'''

In [ ]:
'''preprocessor = ColumnTransformer(
    transformers=[
        ('txt', text_transformer, text_cols)
])
model = MultinomialNB()
clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                     ])
'''

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer()),
('clf', MultinomialNB()),
])

In [ ]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",...
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itse

In [ ]:
preds = text_clf.predict(X_test)
score = metrics.accuracy_score(y_test, preds)
score

0.9063488202383848

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer()),
('clf', RandomForestClassifier(n_estimators=100)),
])

In [ ]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
 

In [ ]:
preds = text_clf.predict(X_test)
score = metrics.accuracy_score(y_test, preds)
score

0.9574312819265386

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
('vect', CountVectorizer(stop_words = stopwords_list)),
('tfidf', TfidfTransformer()),
('clf', LogisticRegression()),
])

In [ ]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",...
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight

In [ ]:
preds = text_clf.predict(X_test)
score = metrics.accuracy_score(y_test, preds)
score

0.9496472877645342